<a href="https://colab.research.google.com/github/mwl10/hetvae/blob/errors/src/catalina_optuna.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import shutil
os.chdir('/content')
! git clone --branch errors https://github.com/mwl10/hetvae
os.chdir('/content/hetvae')
! pip install -r requirements.txt
os.chdir('/content/hetvae/src')
import numpy as np
import torch
import optuna
from optuna.trial import TrialState
import torch.optim as optim
import models
from argparse import Namespace
import torch.optim as optim
import utils
import my_utils
import pandas as pd
from glob import glob
import importlib
import vae_models
import matplotlib.pyplot as plt
import sys
from dataset import DataSet
%matplotlib inline

Cloning into 'hetvae'...
remote: Enumerating objects: 4039, done.
remote: Counting objects: 100% (1075/1075), done.
remote: Compressing objects: 100% (889/889), done.
remote: Total 4039 (delta 247), reused 293 (delta 186), pack-reused 2964
Receiving objects: 100% (4039/4039), 46.21 MiB | 31.07 MiB/s, done.
Resolving deltas: 100% (1192/1192), done.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 210 kB 5.0 MB/s 
     |████████████████████████████████| 81 kB 7.9 MB/s 
     |████████████████████████████████| 149 kB 71.1 MB/s 
     |████████████████████████████████| 78 kB 8.6 MB/s 
     |████████████████████████████████| 15.7 MB 81.8 MB/s 
     |████████████████████████████████| 308 kB 98.8 MB/s 
     |████████████████████████████████| 112 kB 96.1 MB/s 
     |████████████████████████████████| 2.1 MB 67.0 MB/s 
     |████████████████████████████████| 596 kB 62.8 MB/s 
     |███████████████████████████

In [3]:
def define_model_args(trial):

    args = Namespace(
        batch_size = 8, #trial.suggest_categorical("batch_size", [8,16,32]),
        bound_variance = True,
        const_var = False,
        dataset='toy',
        dropout = 0.3,#trial.suggest_float("dropout", 0.0,0.5),
        elbo_weight = trial.suggest_float("elbo_weight", 0.0, 2.0),
        embed_time = trial.suggest_categorical("embed_time", [16,32,64]),
        enc_num_heads=trial.suggest_categorical("enc_num_heads", [4,8,16]),
        intensity=True,
        k_iwae=1,
        kl_annealing=False,#trial.suggest_categorical("kl_annealing",False),
        kl_zero=False, 
        latent_dim=trial.suggest_categorical("latent_dim", [64,128]),
        lr=trial.suggest_float("lr", 1e-7, 1e-1, log=True),
        mixing="concat_and_mix",#trial.suggest_categorical("mixing", ["concat", "concat_and_mix"]),#"separate", "interp_only", "na"]),
        mse_weight=trial.suggest_float("mse_weight",1,20),
        #n=trial.suggest_categorical("n", [8,16,32,64,128]),
        net='hetvae', 
        niters=1000, 
        norm=True, 
        normalize_input='znorm', 
        num_ref_points=128#trial.suggest_categorical("num_ref_points", [64, 128,256]),
        rec_hidden=trial.suggest_categorical("rec_hidden", [32,64, 128]),
        recon_loss=False, 
        sample_tp= trial.suggest_float("sample_tp", 0.1,0.9), # will be ignored
        save=False, 
        seed=0, 
        shuffle=True, 
        std=0.1, 
        var_per_dim=False, 
        width=trial.suggest_categorical("width", [128,256])
    )

    return args

In [4]:
EPOCHS = 500
#FILES = glob('/content/hetvae/data/CAT/*/*')
#FILES = glob('/content/hetvae/data/CAT/*/*')[:50]

FILES = glob('/content/hetvae/data/CAT/*/*')[:2]
DIM = 1

In [5]:
from contextlib import contextmanager
import sys, os

@contextmanager
def suppress_stdout():
    with open(os.devnull, "w") as devnull:
        old_stdout = sys.stdout
        sys.stdout = devnull
        try:  
            yield
        finally:
            sys.stdout = old_stdout

In [6]:
def objective(trial):

    args = define_model_args(trial)

    seed = args.seed
    torch.manual_seed(seed)
    np.random.seed(seed)
    torch.cuda.manual_seed(seed)
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    # dataset hyperparams
    num_samples=trial.suggest_int("num_samples", 1,6)
    # normalize choices for optuna
    #x_by_range = trial.suggest_categorical("x_by_range", [True, False])
    #y_by_range = False #trial.suggest_categorical("y_by_range", [True, False])
    #normalize_y = trial.suggest_categorical("normalize_y", ["all", "individual"])

    with suppress_stdout():
        # AGN = DataSet().add_files(FILES).files_to_numpy().handle_dups().prune_graham().resample_dataset(num_samples=num_samples) \
        #     .normalize(y_by_range=y_by_range, x_by_range=x_by_range).set_union_x().zero_fill().make_masks(frac=args.sample_tp)
        # AGN = DataSet()
        # AGN.dataset = lcs
        # AGN = AGN.handle_dups().prune_outliers().resample_dataset(num_samples=num_samples) \
        #      .normalize(y_by_range=y_by_range, x_by_range=x_by_range).set_union_x().zero_fill().make_masks(frac=args.sample_tp)
        Catalina = DataSet() \
                .add_files(FILES) \
                .files_to_numpy() \
                .handle_dups() \
                .prune_graham(plot=False, index=10, std_threshold=1) \
                .resample_dataset(num_samples=3) \
                .normalize(normalize_x = 'no', x_by_range=False) \
                .reorder() \
                .set_union_x() \
                .zero_fill() \
                .error_to_sample_weight()


    LightCurves = Catalina.dataset

    training, valid = np.split(LightCurves, [int(np.floor(1*len(LightCurves)))])# shuffle?
    
    
    train_loader = torch.utils.data.DataLoader(training, batch_size=args.batch_size)
    valid_loader = torch.utils.data.DataLoader(valid, batch_size=args.batch_size)
    
    
    net = models.load_network(args, DIM, torch.Tensor(Catalina.union_x)) # , device="cuda"
    

    optimizer_name = trial.suggest_categorical("optimizer", ["Adam"])
    optimizer = getattr(optim, optimizer_name)(net.parameters(), lr=args.lr)
    frac = 0.4#trial.suggest_float("sample_tp", 0.1,0.9)
    step_size = trial.suggest_categorical("step_size", [200,300,400,500])
    gamma = trial.suggest_float("gamma",0,1)
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, gamma=gamma, step_size=step_size)
    beta = trial.suggest_float("beta", 0,1)
    for epoch in range(EPOCHS):
        nll_loss, mse = my_utils.train(net, optimizer, epoch, train_loader, args, device=device, frac=frac, beta=beta)
        #nll_loss = my_utils.evaluate(net, valid_loader, device=device)
        trial.report(nll_loss, epoch)
        if trial.should_prune():
            raise optuna.exceptions.TrialPruned()

    return nll_loss

In [7]:
study = optuna.create_study(direction="minimize")

[I 2022-06-23 08:15:37,826] A new study created in memory with name: no-name-778f6f5f-a22c-4908-8f5a-a280e57df431


In [8]:
study.optimize(objective, n_trials=100, timeout=600)

pruned_trials = study.get_trials(deepcopy=False, states=[TrialState.PRUNED])
complete_trials = study.get_trials(deepcopy=False, states=[TrialState.COMPLETE])

print("Study statistics: ")
print("  Number of finished trials: ", len(study.trials))
print("  Number of pruned trials: ", len(pruned_trials))
print("  Number of complete trials: ", len(complete_trials))

print("Best trial:")
trial = study.best_trial

print("  Value: ", trial.value)

print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

/content/hetvae/src/dataset.py:328: RuntimeWarning: divide by zero encountered in true_divide
  self.dataset[:,:,2] = 1. / self.dataset[:,:,2]


Iter: 0, train loss: 25.3136, avg nll: 1.5712, avg kl: 7.2040, mse: 0.968706, mae: 0.788234
Iter: 100, train loss: 13.5077, avg nll: 1.4596, avg kl: 0.0049, mse: 0.998239, mae: 0.820888
Iter: 200, train loss: 13.6465, avg nll: 1.4212, avg kl: 0.0062, mse: 1.017059, mae: 0.824017
Iter: 300, train loss: 12.5543, avg nll: 1.3625, avg kl: 0.0067, mse: 0.926468, mae: 0.786991
Iter: 400, train loss: 13.4512, avg nll: 1.4287, avg kl: 0.0061, mse: 0.997931, mae: 0.810043


[I 2022-06-23 08:16:37,417] Trial 0 finished with value: 1.4359208345413208 and parameters: {'elbo_weight': 1.788725435572976, 'embed_time': 16, 'enc_num_heads': 8, 'latent_dim': 32, 'lr': 0.0018109242465550776, 'mse_weight': 10.908011412354638, 'num_ref_points': 256, 'rec_hidden': 128, 'sample_tp': 0.46331654711579984, 'width': 256, 'num_samples': 3, 'optimizer': 'Adam', 'step_size': 200, 'gamma': 0.8782403593597651, 'beta': 0.9263155894762508}. Best is trial 0 with value: 1.4359208345413208.
/content/hetvae/src/dataset.py:328: RuntimeWarning: divide by zero encountered in true_divide
  self.dataset[:,:,2] = 1. / self.dataset[:,:,2]


Iter: 0, train loss: 4.0759, avg nll: 1.5769, avg kl: 0.8412, mse: 0.969767, mae: 0.796417
Iter: 100, train loss: 3.5741, avg nll: 1.4598, avg kl: 0.0040, mse: 0.998377, mae: 0.821017
Iter: 200, train loss: 3.5482, avg nll: 1.4149, avg kl: 0.0074, mse: 1.017097, mae: 0.823402
Iter: 300, train loss: 3.3307, avg nll: 1.3615, avg kl: 0.0026, mse: 0.929827, mae: 0.788144
Iter: 400, train loss: 3.5384, avg nll: 1.4299, avg kl: 0.0009, mse: 1.001287, mae: 0.810887


[I 2022-06-23 08:16:52,233] Trial 1 finished with value: 1.4322580099105835 and parameters: {'elbo_weight': 1.3060908332374406, 'embed_time': 32, 'enc_num_heads': 8, 'latent_dim': 32, 'lr': 0.0005523697027954683, 'mse_weight': 1.6682566076390137, 'num_ref_points': 64, 'rec_hidden': 128, 'sample_tp': 0.6362539135959181, 'width': 128, 'num_samples': 4, 'optimizer': 'Adam', 'step_size': 500, 'gamma': 0.6443410820064809, 'beta': 0.36277203717784456}. Best is trial 1 with value: 1.4322580099105835.
/content/hetvae/src/dataset.py:328: RuntimeWarning: divide by zero encountered in true_divide
  self.dataset[:,:,2] = 1. / self.dataset[:,:,2]


...

Iter: 0, train loss: 17.9679, avg nll: 1.6994, avg kl: 2.5405, mse: 0.964793, mae: 0.789857
Iter: 0, train loss: 13.8713, avg nll: 1.6085, avg kl: 5.2166, mse: 0.965822, mae: 0.789152
Iter: 0, train loss: 8.1529, avg nll: 1.5340, avg kl: 1.8911, mse: 0.966394, mae: 0.789435


/content/hetvae/src/dataset.py:328: RuntimeWarning: divide by zero encountered in true_divide
  self.dataset[:,:,2] = 1. / self.dataset[:,:,2]
[I 2022-06-23 08:18:11,815] Trial 11 pruned. 
/content/hetvae/src/dataset.py:328: RuntimeWarning: divide by zero encountered in true_divide
  self.dataset[:,:,2] = 1. / self.dataset[:,:,2]
[I 2022-06-23 08:18:11,955] Trial 12 pruned. 
/content/hetvae/src/dataset.py:328: RuntimeWarning: divide by zero encountered in true_divide
  self.dataset[:,:,2] = 1. / self.dataset[:,:,2]
[I 2022-06-23 08:18:12,020] Trial 13 pruned. 


Iter: 0, train loss: 17.3267, avg nll: 1.5340, avg kl: 1.8911, mse: 0.966394, mae: 0.789435
Iter: 0, train loss: 13.9066, avg nll: 1.6582, avg kl: 0.9782, mse: 0.969561, mae: 0.796351


/content/hetvae/src/dataset.py:328: RuntimeWarning: divide by zero encountered in true_divide
  self.dataset[:,:,2] = 1. / self.dataset[:,:,2]
[W 2022-06-23 08:18:12,320] Trial 14 failed because of the following error: RuntimeError('CUDA out of memory. Tried to allocate 2.59 GiB (GPU 0; 15.90 GiB total capacity; 10.39 GiB already allocated; 1.99 GiB free; 12.99 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF')
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/optuna/study/_optimize.py", line 213, in _run_trial
    value_or_values = func(trial)
  File "<ipython-input-6-cdd22d0f5542>", line 48, in objective
    nll_loss, mse = my_utils.train(net, optimizer, epoch, train_loader, args, device=device, frac=frac, beta=beta)
  File "/content/hetvae/src/my_utils.py", line 56, in train
    loss_info.composite_loss.

RuntimeError: ignored

In [ ]:
# [I 2022-06-17 16:55:13,210] Trial 3 finished with value: 0.09719855338335037 and parameters: {'dropout': 0.02418752335062102, 'elbo_weight': 1.1061943368326204, 'embed_time': 16, 'lr': 4.212141191372822e-05, 'mse_weight': 5.108416562407145, 'rec_hidden': 128, 'num_samples': 6, 'x_by_range': True, 'optimizer': 'Adam', 'sample_tp': 0.1979453313080004}. Best is trial 3 with value: 0.09719855338335037.

optuna.visualization.plot_param_importances(study)

In [ ]:

optuna.visualization.plot_optimization_history(study)

In [ ]:
optuna.visualization.plot_slice(study, params=["optimizer"])